In [65]:
import os
from dotenv import load_dotenv
load_dotenv(encoding='utf-8')

True

# RAG pipeline

## Vector Store and Retriever

In [66]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from pymongo import MongoClient
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere
from llama_index.postprocessor.flag_embedding_reranker import (
    FlagEmbeddingReranker,
)
from llama_index.core.postprocessor import LLMRerank
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.llms import OpenAI
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder


In [67]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL")) # DEFAULT_OPENAI_MODEL='gpt-4o-mini-2024-07-18'
llm = ChatOpenAI(model="gpt-4o-mini") # DEFAULT_OPENAI_MODEL='gpt-4o-mini-2024-07-18'

# embedding_model=OpenAIEmbeddings(model=os.getenv("DEFAULT_OPENAI_EMBEDDING"), disallowed_special=())
embedding_model=OpenAIEmbeddings(disallowed_special=())

In [148]:
# Define MongoDB vector database
client = MongoClient(os.getenv("MONGO_DB_URL"))
db_name = "fellowshipai"
collection_name = "enterprise_data_2"
atlas_collection = client[db_name][collection_name]
index_name = "vector_index_erp_2"
compression_retriever_model = "cohere"

In [134]:
# Define vector store and retriever
def get_vector_store_retriver(index_name, embedding_model, collection):

  vector_store = MongoDBAtlasVectorSearch(
      embedding = embedding_model,
      collection = atlas_collection,
      index_name = index_name
  )

  retriever = vector_store.as_retriever(
      search_type = "similarity",
      search_kwargs = { "k": 10}  # "score_threshold": 0.75 
  )

  return(vector_store, retriever)

vector_store, retriever = get_vector_store_retriver(index_name, embedding_model, atlas_collection)

In [70]:
# Instantiate Atlas Vector Search as a retriever
retriever = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = { "k": 10  }
 )

In [71]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [72]:
# Get the compressed retriever
from FlagEmbedding import FlagLLMReranker
from flashrank import Ranker, RerankRequest
def get_compressed_retriever(model_name):
    if model_name == "cohere":
        compressor = CohereRerank(model="rerank-english-v3.0")
        compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever
        )
    
    if model_name == "crossEncoderReranker":
        model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
        compressor = CrossEncoderReranker(model=model, top_n=3)
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor, base_retriever=retriever
        )
    
    if model_name == "gptCompressor":
        llm = ChatOpenAI(temperature=0, model='gpt-4')
        compressor = LLMChainExtractor.from_llm(llm)
        compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever
        )

    return compression_retriever

In [73]:
query = "What are the tickets logged at tech innovators inc?"
docs = retriever.invoke(query)
pretty_print_docs(docs)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Document 1:

IntroductionTech Innovators Inc. is committed to ensuring the security and integrity of its on-premise systems. This security policy outlines the measures and procedures to protect these systems from unauthorized access, data breaches, and other security threats. This policy applies to all
----------------------------------------------------------------------------------------------------
Document 2:

by:Jhon Watch, Chief Information Security Officer20/07/2024Effective Date:20/07/2024Tech Innovators Inc.#12, IT parkTech city, Tech state
----------------------------------------------------------------------------------------------------
Document 3:

by:Jhon Watch, Chief Information Security Officer20/07/2024Effective Date:20/07/2024Tech Innovators Inc.#12, IT parkTech city, Tech state
----------------------------------------------------------------------------------------------------
Document 4:

This policy applies to all employees, contractors, and third-party vendors who

In [64]:
compression_retriever = get_compressed_retriever("cohere")
compressed_docs = compression_retriever.invoke(
    "What are the tickets logged at tech innovators inc?"
)
pretty_print_docs(compressed_docs)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"


Document 1:

Ticket ID: TI-67890Created Date: July 20, 2024Priority: HighStatus: OpenRequester InformationName: Emily ClarkDepartment: SalesLocation: United StatesEmail: emily.clark@techinnovators.comPhone: (555) 987-6543Issue SummaryTitle: Unable to access CRM systemDescriptionEmily Clark reported that she is
----------------------------------------------------------------------------------------------------
Document 2:

IntroductionTech Innovators Inc. is committed to ensuring the security and integrity of its on-premise systems. This security policy outlines the measures and procedures to protect these systems from unauthorized access, data breaches, and other security threats. This policy applies to all
----------------------------------------------------------------------------------------------------
Document 3:

IntroductionTech Innovators Inc. is committed to protecting the confidentiality, integrity, and availability of its data and resources hosted on Microsoft Azure. This se

In [62]:
compression_retriever = get_compressed_retriever("crossEncoderReranker")
compressed_docs = compression_retriever.invoke(
    "What are the tickets logged at tech innovators inc?"
)
pretty_print_docs(compressed_docs)

c:\Users\Vadi\anaconda3\envs\toolkit\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vadi\.cache\huggingface\hub\models--BAAI--bge-reranker-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cpu
INFO:httpx:HTTP Request: POS

Document 1:

Ticket ID: TI-67890Created Date: July 20, 2024Priority: HighStatus: OpenRequester InformationName: Emily ClarkDepartment: SalesLocation: United StatesEmail: emily.clark@techinnovators.comPhone: (555) 987-6543Issue SummaryTitle: Unable to access CRM systemDescriptionEmily Clark reported that she is
----------------------------------------------------------------------------------------------------
Document 2:

IntroductionTech Innovators Inc. is committed to ensuring the security and integrity of its on-premise systems. This security policy outlines the measures and procedures to protect these systems from unauthorized access, data breaches, and other security threats. This policy applies to all
----------------------------------------------------------------------------------------------------
Document 3:

IntroductionTech Innovators Inc. is committed to protecting the confidentiality, integrity, and availability of its data and resources hosted on Microsoft Azure. This se

In [21]:
compression_retriever = get_compressed_retriever("gptCompressor")
compressed_docs = compression_retriever.invoke(
    "What are the tickets logged at tech innovators inc?"
)
pretty_print_docs(compressed_docs)

c:\Users\Vadi\anaconda3\envs\toolkit\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\Vadi\anaconda3\envs\toolkit\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\Vadi\anaconda3\envs\toolkit\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\Vadi\anaconda3\envs\toolkit\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\Vadi\anaconda3\envs\toolkit\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an out

Document 1:

Ticket ID: TI-67890Created Date: July 20, 2024Priority: HighStatus: OpenRequester InformationName: Emily ClarkDepartment: SalesLocation: United StatesEmail: emily.clark@techinnovators.comPhone: (555) 987-6543Issue SummaryTitle: Unable to access CRM systemDescriptionEmily Clark reported that she is
----------------------------------------------------------------------------------------------------
Document 2:

Ticket ID: IT-12345Created Date: July 20, 2024Priority: HighStatus: OpenRequester InformationName: John DoeDepartment: MarketingEmail: john.doe@example.comPhone: (555) 123-4567Issue SummaryTitle: Unable to access the Marketing databaseDescriptionJohn Doe reported that he is unable to access the


## RAG Pipeline

In [149]:
compression_retriever_model

'cohere'

In [150]:
# Define a prompt template
import pprint
def call_openai(question):

   question = question['question']

   retriever = vector_store.as_retriever(
      search_type = "similarity",
      search_kwargs = { "k": 10  }
      )

   # Get the compression retriever to rerank and get top 3 documents only
   compression_retriever = get_compressed_retriever(compression_retriever_model)

   preamble = "" # read from cohere front end or use the input to the API
   #question = 
   SAFETY_PREAMBLE = "The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral."
   BASIC_RULES = "You are a powerful conversational AI trained by openAI to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions."
   TASK_CONTEXT = "You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging."
   STYLE_GUIDE = "Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling."
   INSTRUCTIONS = """You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
   You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
               Use the following pieces of context to answer the question at the end.
               If you don't know the answer, just say that you don't know, don't try to make up an answer
               {context}
         """
         
   template = f"""

      {SAFETY_PREAMBLE}
      {BASIC_RULES}
      {TASK_CONTEXT}
      {STYLE_GUIDE}
      {INSTRUCTIONS}

   """
   if preamble:
      template += f"""{preamble}\n\n"""


   template +=  f"""Question: {question}\n\n"""

   custom_rag_prompt = PromptTemplate.from_template(template)

   #llm = get_llm_model("openai")
   # llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))
   llm = ChatOpenAI(model="gpt-4o-mini")
   
   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": compression_retriever | format_docs, "question": RunnablePassthrough()}  #Insted of retriever add compression_retriever here
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever.invoke(question)


   return{
      'answer': answer,
      'contexts': [str(doc) for doc in similar]
      }

In [18]:
import requests
import cohere
import os
import json
os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

def call_cohere(question):
  # Define the API endpoint for streaming
  url = "http://localhost:8000/v1/chat"
  bearer = os.getenv('BEARER_SECRET_KEY')

  # Set headers
  headers = {
      "User-Id": "me",
      "Content-Type": "application/json",
      "Authorization": f"Bearer {bearer}",
      "Cohere-Stream": "true",  # Enable streaming for chatbot responses
  }

  # Set the message to send
  message = question['question']

  # Create the payload as a JSON dictionary
  data = {"message": message}

  # Send the POST request using requests
  response = requests.post(url, headers=headers, json=data)

  # Check for successful response
  if response.status_code == 200:
    # Handle streaming response
    for line in response.iter_lines():
      # Decode the response (if necessary)
      decoded_line = line.decode("utf-8")
      # Process the received data from the stream (print it here)
      resp = json.loads(decoded_line)
      return{
        'answer': resp['chat_history'][1]['message'],
        'contexts': "None"
        }
  else:
    return{
        'answer': {response.status_code},
        'contexts': "None"
    }

### Test the RAG pipeline

In [151]:
# Test sample
question = {'question': "What are the steps to install and configure the Tech Innovator Vector Database?"}
answer = call_openai(question)
print(answer['answer'][:150])

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


To install and configure the Tech Innovator Vector Database, follow these steps:

1. **Prerequisites**:
   - Ensure you have administrative access to 


In [15]:
# # Test sample Cohere
# question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
# answer = call_cohere(question)
# print(answer['answer'][:150])

In [152]:
print(answer['contexts'])

["page_content='steps.Check the log files located in /var/log/vector_db/ for error messages.ConclusionYou have successfully installed and configured the Tech Innovator Vector Database. You can now start using it for your applications.For more information, visit the official documentation.' metadata={'_id': {'$oid': '66dffd0adee1cfdeccbeca08'}, 'pageid': '819215', 'department': 'IT', 'title': 'Tech Innovator Vector Database Installation Guide'}", "page_content='PrerequisitesEnsure you have administrative access to the server where the database will be installed.Verify that your system meets the minimum hardware and software requirements.Install the latest version of Java Development Kit (JDK).Step 1: Download the InstallerVisit the Tech Innovator Vector' metadata={'_id': {'$oid': '66dffd0adee1cfdeccbeca02'}, 'pageid': '819215', 'department': 'IT', 'title': 'Tech Innovator Vector Database Installation Guide'}", "page_content='the Tech Innovator Vector Database download page.Select the ap

# RAG pipeline evaluation

## Test data set prep

In [161]:
import pandas as pd
import json

def json_to_dataframe(json_file_path):
  """Reads a JSON file and converts it to a pandas DataFrame.

  Args:
    json_file_path (str): The path to the JSON file.

  Returns:
    pandas.DataFrame: The DataFrame created from the JSON data.
  """

  with open(json_file_path, 'r') as f:
    data = json.load(f)

  # Handle different JSON structures
  if isinstance(data, list):
    # If the JSON data is a list of dictionaries, create a DataFrame directly
    df = pd.DataFrame(data)
  elif isinstance(data, dict):
    # If the JSON data is a single dictionary, convert it to a list of dictionaries
    df = pd.DataFrame([data])
  else:
    raise ValueError("Unsupported JSON structure")

  return df

In [162]:
# Example usage:
from from_root import from_root
import os
folder = "data/test_dataset/test_dataset_it.json"
# json_file_path = os.path.join(from_root(), folder)
# data_to_upload = json_to_dataframe(json_file_path)
data_to_upload = json_to_dataframe(folder)

## RAGAS evaluation

In [ ]:
# 

In [163]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers = []
for question in data_to_upload['question']:
    question_dict = {'question': question}
    answer = call_openai(question_dict)
    answers.append(answer['answer'])

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/emb

In [113]:
# cohere_answers = []
# for question in data_to_upload['question']:
#     question_dict = {'question': question}
#     answer = call_cohere(question_dict)
#     cohere_answers.append(answer['answer'])
# cohere_answers = [
#     "To install and configure the Tech Innovator Vector Database, you must first download the installer from the Tech Innovator Vector Database download page. Then, select the appropriate version for your operating system and download it.",
#     "I'm sorry, but I can't find the specific error message that John Doe received when attempting to access the Marketing database. All I can tell you is that he was unable to access it from his workstation.",
#     "Tech Innovators Inc. is committed to protecting the confidentiality, integrity and availability of its data and resources hosted on Microsoft Azure.",
#     "The settings that need to be updated in the configuration file for the Tech Innovator Vector Database are the database port and the port number for the database server.",
#     "The mission of Tech Innovators Inc. is to drive innovation and deliver software development, IT consulting, and digital transformation services.",
#     "I'm sorry, I can't find any information about the strategies Tech Innovators Inc. uses to protect its Azure and on-premise systems.",
#     "To install the Tech Innovator Vector Database on a compatible operating system, you need to have the latest version of the Java Development Kit (JDK).",
#     "To install the Tech Innovator CV Parser, you need to:1. Choose the version compatible with your operating system.2. Click Download. 3. Locate the downloaded installer file in your Downloads.",
#     "The commands to start and enable the Tech Innovator Vector DB service are: 1.`sudo systemctl start vector_db` 2. `sudo systemctl enable vector_db`"
# ]

In [114]:
# # Prepare data for Cohere
# data_to_upload_cohere = data_to_upload[:]

In [164]:
# update the dataset with answers
data_to_upload['answers'] = answers

In [115]:
# data_to_upload_cohere['answers'] = cohere_answers

In [165]:
from datasets import Dataset

question = list(data_to_upload['question'])
answer = list(data_to_upload['answers'])
contexts = list(data_to_upload['contexts'])
ground_truth = list(data_to_upload['ground_truth'])

data_samples = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset = Dataset.from_dict(data_samples)

In [88]:
# Change for Cohere
# from datasets import Dataset

# question = list(data_to_upload_cohere['question'])
# answer = list(data_to_upload_cohere['answers'])
# contexts = list(data_to_upload_cohere['contexts'])
# ground_truth = list(data_to_upload_cohere['ground_truth'])

# data_samples_cohere = {
#     'question': question,
#     'answer': answer,
#     'contexts': contexts,
#     'ground_truth': ground_truth
# }

# dataset_cohere = Dataset.from_dict(data_samples_cohere)

In [166]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result = evaluate(
    dataset,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Evaluating:   3%|▎         | 1/36 [00:01<00:44,  1.28s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 20

{'answer_relevancy': 0.7609, 'faithfulness': 0.7630, 'context_recall': 1.0000, 'context_precision': 1.0000}

In [116]:
# Evaluate Cohere
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
# from ragas.metrics import (
#     answer_relevancy,
#     faithfulness,
#     context_recall,
#     context_precision,
# )
# result = evaluate(
#     dataset_cohere,
#     metrics=[
#         answer_relevancy,
#         faithfulness,
#         context_recall,
#         context_precision,
#     ],
# )

# result

In [167]:
answers

['To install and configure the Tech Innovator Vector Database, follow these steps:\n\n1. **Prerequisites**:\n   - Ensure you have administrative access to the server where the database will be installed.\n   - Verify that your system meets the minimum hardware and software requirements.\n   - Install the latest version of the Java Development Kit (JDK).\n\n2. **Step 1: Download the Installer**:\n   - Visit the Tech Innovator Vector Database download page.\n   - Select the appropriate version for your operating system.\n   - Download the installer package to your local machine.\n\n3. **Step 2: Install the Database**:\n   - Open a terminal or command prompt.\n   - Navigate to the directory where the installer package is located.\n   - Run the installer package to initiate the installation process.\n\n4. **Step 3: Configuration**:\n   - Follow any on-screen instructions to complete the configuration process for the database.\n\n5. **Step 4: Check for Errors** (if necessary):\n   - After i

In [117]:
# cohere_answers

In [168]:
df = result.to_pandas()
df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,What are the steps to install and configure th...,To install and configure the Tech Innovator Ve...,[Prerequisites\nEnsure you have administrative...,The steps to install and configure the Tech In...,1.000000,1.000000,1.0,1.0
1,What error message did John Doe receive when a...,"John Doe received the error message: ""Database...",[Ticket ID:\n IT-12345\nCreated Date:\n July 2...,"John Doe received the error message stating, ""...",0.999706,1.000000,1.0,1.0
2,What measures does the security policy of Tech...,The security policy of Tech Innovators Inc. ou...,[Introduction\nTech Innovators Inc. is committ...,The security policy of Tech Innovators Inc. ou...,1.000000,0.733333,1.0,1.0
3,What settings need to be updated in the config...,To update the settings in the configuration fi...,[Prerequisites\nEnsure you have administrative...,The settings that need to be updated in the co...,0.000000,0.500000,1.0,1.0
4,What is the mission of the Tech Innovators Inc...,I don't know the specific mission of the Tech ...,[:check_mark:\natlassian-check_mark\n#F4F5F7\n...,Our mission is to enable and empower our busin...,0.000000,0.000000,1.0,1.0
5,What strategies does Tech Innovators Inc. use ...,Tech Innovators Inc. employs several strategie...,[Introduction\nTech Innovators Inc. is committ...,Tech Innovators Inc. employs several strategie...,0.998669,1.000000,1.0,1.0
6,What’s needed to install the Tech Innovator Ve...,To install the Tech Innovator Vector Database ...,[Prerequisites\nEnsure you have administrative...,To install the Tech Innovator Vector Database ...,0.927728,0.833333,1.0,1.0
7,What are the key system and access needs for i...,To install the Tech Innovator CV Parser and Ve...,[Prerequisites\nBefore you start the installat...,The key system and access needs for installing...,0.961962,0.800000,1.0,1.0
8,What commands start and enable the Tech Innova...,To start and enable the Tech Innovator Vector ...,[Prerequisites\nEnsure you have administrative...,The commands to start and enable the Tech Inno...,0.959930,1.000000,1.0,1.0


In [169]:
df.to_csv('cohere.csv')